In [1]:
"""
Maxwell Zheng 1004907871
Google Colab: https://colab.research.google.com/drive/1rT7p9Tfa9ciVgkNR2WroAuo6xa26feGg?usp=sharing
"""


In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [5]:
# 1a

# unable to upload to Colab so used local file
f = open(r'C:\Users\Maxwell\Documents\Code\Python\APS360\src\Labs\Lab5\SMSSpamCollection')

for line in open(r'C:\Users\Maxwell\Documents\Code\Python\APS360\src\Labs\Lab5\SMSSpamCollection'):
  if 'ham' in line:
    print(line)
    for line in f:
      if 'spam' in line:
        print(line)
        break
    break
    

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



In [8]:
# 1b

i = 0
j = 0
with open('SMSSpamCollection') as f:
    for line in f:
        if 'spam' in line:
          i += 1
        else:
          j += 1
        
    print("# spam messages: ",i)
    print("# non-spam messages: ",j)


# spam messages:  747
# non-spam messages:  4827


In [ ]:
# 1c

"""
Advatanges:
    more accurate
    can detect punctation as well as characters
Disadvantages:
    needs more data
    longer processing time
"""

In [10]:
# 1d

import torchtext

text_field = torchtext.data.Field(sequential=True,      
                                  tokenize=lambda x: x, 
                                  include_lengths=True,                                   
                                  batch_first=True,
                                  use_vocab=True)  

label_field = torchtext.data.Field(sequential=False,    
                                   use_vocab=False,     
                                   is_target=True,      
                                   batch_first=True,
                                   preprocessing=lambda x: int(x == 'spam')) 

fields = [('label', label_field), ('sms', text_field)]
dataset = torchtext.data.TabularDataset("SMSSpamCollection", "tsv", fields)

print(dataset[0].sms)
print(dataset[0].label)

train, valid, test = dataset.split(split_ratio = [0.6,0.2,0.2], stratified=False, strata_field='label', random_state=None)

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
0


In [ ]:
# 1e

"""
A balanced training set is helpful for training a nerual net because it sees both spam and non-spam
and is able to distinguish between the two.
It is important to have sample sizes to obtain good accuracy.
"""

In [11]:
old_train_examples = train.examples

train_spam = []
for item in train.examples:
    if item.label == 1:
        train_spam.append(item)

train.examples = old_train_examples + train_spam * 6

In [ ]:
# 1f

"""
vocab.stoi maps token strs to numerical ids
vocab.itos is a list of token strs index by their numerical id
"""

In [12]:
text_field.build_vocab(train)
text_field.build_vocab(valid)
label_field.build_vocab(train)
label_field.build_vocab(valid)
text_field.vocab.stoi
text_field.vocab.itos

['<unk>',
 '<pad>',
 ' ',
 'e',
 'o',
 't',
 'a',
 'n',
 'i',
 's',
 'r',
 'l',
 'h',
 '.',
 'u',
 'd',
 'y',
 'm',
 'g',
 'c',
 'w',
 'p',
 'f',
 'b',
 'k',
 'I',
 'v',
 '0',
 'T',
 ',',
 'S',
 "'",
 'O',
 '1',
 'A',
 'E',
 '?',
 'N',
 '2',
 'H',
 '!',
 'W',
 'C',
 '8',
 '5',
 'x',
 'U',
 'Y',
 'D',
 'R',
 'M',
 '4',
 '&',
 'G',
 'L',
 'P',
 'B',
 '7',
 ';',
 '6',
 '3',
 ':',
 'j',
 '9',
 '-',
 'F',
 'z',
 ')',
 '*',
 'K',
 '/',
 '#',
 'V',
 'J',
 '£',
 'X',
 '"',
 'q',
 '+',
 'ü',
 '(',
 'Q',
 'Ü',
 '=',
 '@',
 '‘',
 '\x92',
 '$',
 '>',
 '…',
 '%',
 '<',
 'Z',
 '\x94',
 '|',
 '~',
 '\x93',
 '»']

In [ ]:
# 1g

"""
unk and pad represent sepcial tokens that will be prepended to the vocab 
unk: out of vocab works that are unknown
pad: padding
"""

In [55]:
# 1h

trainIter = torchtext.data.BucketIterator(train,batch_size=32,sort_key=lambda x: len(x.sms), sort_within_batch=True, repeat=False)                  

In [ ]:
"""
The max length of input is 61
To obtain the number of pad tokens we count the occurence of 1s in the first tensor
"""

In [14]:
x = 0
for batch in train_iter:
    x = x + 1
    print(len(batch))
    print(batch.sms) 
    print(batch.label)
    if x == 10:
        break


32
(tensor([[37,  4,  5,  ...,  7, 36,  2],
        [39,  3,  2,  ...,  6, 10, 36],
        [55, 49, 25,  ..., 70, 27, 51],
        ...,
        [55, 11,  3,  ...,  3, 40,  1],
        [46,  2, 12,  ..., 27, 33,  1],
        [55, 11,  3,  ...,  3, 40,  1]]), tensor([148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148,
        148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148,
        148, 147, 147, 147]))
tensor([0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 1, 0, 1, 1, 1, 1])
32
(tensor([[41, 32, 28,  2, 46,  2, 46, 55,  2, 38,  2, 73, 36],
        [42,  6,  7,  2, 15,  4,  2, 11,  4, 10, 13, 13, 13],
        [32, 24, 13, 13, 13,  2, 42,  2, 16,  6, 13, 13, 13],
        [38, 70, 38,  2, 33, 51, 59,  5, 22, 33, 44, 27, 21],
        [32, 24,  2,  5, 12,  6,  7, 45, 13, 13, 13,  1,  1],
        [25, 31, 17,  2, 15,  4,  7,  3, 13, 13, 13,  1,  1],
        [28, 12,  3,  7,  2,  8,  2, 23, 14, 16, 13,  1,  1],
    

        30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 29]))
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
32
(tensor([[34,  8, 16,  ..., 19, 24, 36],
        [39,  3, 16,  ..., 15,  6, 13],
        [48,  6, 10,  ...,  6, 10, 36],
        ...,
        [39,  8,  2,  ...,  5, 13,  1],
        [41, 12,  6,  ..., 15, 36,  1],
        [47,  4, 14,  ..., 57, 60,  1]]), tensor([48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 47, 47, 47, 47,
        47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47]))
tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1])
32
(tensor([[65, 10,  3,  ...,  3,  7, 15],
        [49,  3, 21,  ..., 44, 27, 21],
        [65,  8,  7,  ...,  2, 33, 59],
        ...,
        [46, 10, 18,  ..., 60, 34, 73],
        [46, 10, 18,  ..., 57, 47, 65],
        [55, 11,  3,  ..., 15, 16, 13]]), tensor([159, 159, 159, 159, 159, 159, 159, 1

In [15]:
# 2

import torch
ident = torch.eye(50)
print(ident[0]) 
print(ident[1]) 
x = torch.tensor([[1, 2], [3, 4]])
print(ident[x]) 

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        

In [61]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms

In [62]:
class SpamRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(SpamRNN, self).__init__()
        self.ident = torch.eye(input_size)
        print(torch.eye(input_size))
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, inp):

        print(inp)
        inp = self.ident[inp]
        h0 = torch.zeros(1, inp.size(0), self.hidden_size)
        out, _ = self.rnn(inp)
        out = torch.cat([torch.max(out, dim=1)[0], torch.mean(out, dim=1)], dim=1)
        self.fc(out)
        
        return out

In [23]:
# 3a

trainIter = torchtext.data.BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.sms), sort_within_batch=True, repeat=False)                  

In [21]:
def getAccuracy(model, data):
    
    correct, total = 0, 0
    for sms, labels in data:
        output = model(sms)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
        
    return correct / total

In [46]:
# 3b

trainLoader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
valLoader = torch.utils.data.DataLoader(valid, batch_size=16, shuffle=True)
testLoader = torch.utils.data.DataLoader(test, batch_size=16, shuffle=True)

In [28]:
def train_rnn_network(model, train, valid, numEpochs=5, learningRate=1e-5):
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
    losses, trainAcc, valAcc = [], [], []
    epochs = []
    for epoch in range(numEpochs):
        transform = transforms.Compose([transforms.ToTensor()])
        for sms, labels in train:
            optimizer.zero_grad()
            pred = model(sms)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
        losses.append(float(loss))

        epochs.append(epoch)
        trainAcc.append(get_accuracy(model, trainLoader))
        valAcc.append(get_accuracy(model, valLoader))
        print("Epoch %d; Loss %f; Train Acc %f; Val Acc %f" % (epoch+1, loss, traiAcc[-1], valAcc[-1]))

    plt.title("Training Curve")
    plt.plot(losses, label="Train")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochs, train_acc, label="Train")
    plt.plot(epochs, valid_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

In [29]:
spam = SpamRNN(50, 50, 2)
train_rnn_network(spam, trainLoader, valLoader, numEpochs=5, learningRate=1e-05)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torchtext.data.example.Example'>

In [33]:
# 3c

"""
I will choose learning rate, epoch, batch size and hidden size
"""

In [ ]:
spam = SpamRNN(50, 50, 2)
train_rnn_network(spam, trainLoader, valLoader, numEpochs=5, learningRate=0.001)

In [ ]:
spam = SpamRNN(50, 50, 2)
train_rnn_network(spam, trainLoader, valLoader, numEpochs=5, learningRate=0.01)

In [ ]:
spam = SpamRNN(50, 50, 2)
train_rnn_network(spam, trainLoader, valLoader, numEpochs=30, learningRate=0.01)

In [ ]:
spam = SpamRNN(50, 50, 2)
train_rnn_network(spam, trainLoader, valLoader, numEpochs=30, batch_size=64, learningRate=0.01)